In [1]:
!pip install -q transformers accelerate torch sentencepiece gradio


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

print("Model loaded successfully")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded successfully


In [1]:
def explain_medical_text(medical_text):
    if not medical_text.strip():
        return ""

    prompt = f"""
You are a healthcare communication assistant.

Your ONLY task is to interpret medical text in plain language.
You must NOT provide guidance, advice, or recommendations.

Strict rules:
- Explain what the medical text is referring to, in general terms
- Explain why doctors commonly record or mention this information
- Do NOT suggest actions, treatments, medications, lifestyle changes, or next steps
- Do NOT tell the patient what they should or could do
- Do NOT provide medical advice or diagnosis
- Do NOT repeat the original wording
- Use neutral, informational language only
- Write 4–6 short sentences

Medical text:
{medical_text}

Plain-language explanation:
"""

    output = pipe(
        prompt,
        max_new_tokens=220,
        temperature=0.6,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    explanation = output[0]["generated_text"].split("Plain-language explanation:")[-1].strip()

    return (
        explanation
        + "\n\n—\n"
        + "⚠️ This explanation is generated for educational purposes only. "
        + "It is not a medical diagnosis or medical advice."
    )


In [ ]:
import gradio as gr

with gr.Blocks(css="""
body {
    background-color: #0f172a;
}
.gradio-container {
    font-family: Arial, sans-serif;
}
.card {
    background-color: #1e293b;
    border-radius: 12px;
    padding: 20px;
}
""") as demo:

    gr.Markdown(
        """
        # 🩺 Medical Text Explainer
        **Clear, patient-friendly explanations for medical reports and prescriptions**

        This tool helps patients understand medical text in simple language.
        It is privacy-first and designed for educational use only.
        """
    )

    with gr.Row():
        with gr.Column():
            medical_input = gr.Textbox(
                label="Medical Text",
                placeholder="Paste a medical report or prescription note here",
                lines=8
            )

            explain_btn = gr.Button("Explain clearly")

            clear_btn = gr.Button("Clear")

        with gr.Column():
            output = gr.Textbox(
                label="Patient-Friendly Explanation",
                lines=14,
                interactive=False
            )

    def clear_all():
        return "", ""

    explain_btn.click(
        fn=explain_medical_text,
        inputs=medical_input,
        outputs=output
    )

    clear_btn.click(
        fn=clear_all,
        inputs=[],
        outputs=[medical_input, output]
    )

demo.launch(share=True, debug=True)



/tmp/ipython-input-3396693628.py:3: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css="""


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7c9772cd6f2714d380.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
